## Analysis of Hyatt Dataset on GPU

The dataset consists of 1.5 Miliion records, feedbacks from customers staying at Hyatt Hotels all across the world. Each feedback is mapped with demographics of customer, dates & days of visit, purpose, ratings of each facility they are provided with, etc. The target variable used for prediction is 'Likelihood_to_recommend' which describes on a scale of 1-10, how likely is the customer going to recommend Hyatt Hotel to someone else.

In [1]:
#Loading Packages
#!/usr/bin/env
import sys
import os.path
from pprint import pprint
from time import time
import warnings
warnings.filterwarnings("ignore")

start_time = time()

PWD = !pwd  #set cwd

In [2]:
mapd_path = os.path.join(PWD[0],'..','thirdparty')
sys.path.append(mapd_path)
mapd_path

'/raidStorage/wamsi/mapd-ml/notebooks/../thirdparty'

In [3]:
pygdf_path = os.path.join(PWD[0],'..')
sys.path.append(pygdf_path)
pygdf_path

'/raidStorage/wamsi/mapd-ml/notebooks/..'

Loading PYGDF library

In [4]:
import pygdf

Importing Packages for MapD connection

In [5]:
from mapd import MapD
from mapd import ttypes

In [6]:
#Importing thrift libraries
from thrift.protocol import TBinaryProtocol
from thrift.protocol import TJSONProtocol
from thrift.transport import TSocket
from thrift.transport import THttpClient
from thrift.transport import TTransport

### Setting up MapD connection

In [7]:
def get_client(host_or_uri, port, http):
  if http:
    transport = THttpClient.THttpClient(host_or_uri)
    protocol = TJSONProtocol.TJSONProtocol(transport)
  else:
    socket = TSocket.TSocket(host_or_uri, port)
    transport = TTransport.TBufferedTransport(socket)
    protocol = TBinaryProtocol.TBinaryProtocol(transport)

  client = MapD.Client(protocol)
  transport.open()
  return client

In [8]:
db_name = 'mapd'
user_name = 'mapd'
passwd = 'HyperInteractive'
hostname = 'bewdy.mapd.com'
portno = '9998'

client = get_client(hostname, portno, False)
session = client.connect(user_name, passwd, db_name)
print('Connection Established')

Connection Established


Define Function to convert MapD Query Result into Dictionary

In [9]:
def getDict(results):
    
    try:
        assert results.row_set.is_columnar == True
    except AssertionError:
        print('Please use columns not rows in arguments')
    
    results_dict = {}  #Initialize Dictionary
    
    nrows = len(results.row_set.columns[0].nulls)
    ncols = len(results.row_set.row_desc)
    
    for i in list(range(0,ncols)):
        row_vals = []
        dt = False
        col_typ = ttypes.TDatumType._VALUES_TO_NAMES[results.row_set.row_desc[i].col_type.type]
        col_name = results.row_set.row_desc[i].col_name
        col_arr = results.row_set.row_desc[i].col_type.is_array
        
        if not col_arr:
            if col_typ in ['SMALLINT', 'INT', 'BIGINT', 'TIME', 'TIMESTAMP', 'DATE', 'BOOL']:
                hold = results.row_set.columns[i].data.int_col
            elif col_typ in ['FLOAT', 'DECIMAL', 'DOUBLE']:
                hold = results.row_set.columns[i].data.real_col
            elif col_typ in ['STR']:
                hold = results.row_set.columns[i].data.str_col
            else:
                if col_typ in ['TIME','TIMESTAMP','DATE']:
                  dt = True
                  
                hold = results.row_set.columns[i].data.arr_col
                  
        for j in list(range(0,nrows)):
            if not dt:
                row_vals.append(hold[j])
                #print(hold[j])
            else:
                row_vals.append(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(hold[j])))
        
        results_dict[col_name] = row_vals
    
    return results_dict

### Extracting data using Mapdql

In [10]:
#columns used in query
columns_all = '''Likelihood_Recommend_H,CHECKOUT_HEADER_ID_C,MARKET_CODE_C,MARKET_DESC_C,MARKET_GROUP_C,MAJOR_MARKET_CODE_C,CHANNEL_CODE_C,RATE_PLAN_C,SPLIT_RATE_PLAN_C,CONS_GUEST_ID_C,ROOM_NUM_C,ROOM_TYPE_CODE_C,ROOM_TYPE_DESCRIPTION_C,WALK_IN_FLG_C,RESERVATION_CONFIRMATION_NUM_C,RESERVATION_CONFIRMATION_PAGE_C,RESERVATION_STATUS_C,CHECK_IN_DATE_C,CHECK_OUT_DATE_C,LENGTH_OF_STAY_C,NUMBER_OF_ROOMS_C,ADULT_NUM_C,CHILDREN_NUM_C,POV_CODE_C,QUOTED_RATE_C,PMS_ROOM_REV_C,PMS_ROOM_REV_USD_C,PMS_TOTAL_REV_C,PMS_TOTAL_REV_USD_C,PMS_FOOD_BEVERAGE_REV_C,PMS_FOOD_BEVERAGE_REV_USD_C,PMS_OTHER_REV_C,PMS_OTHER_REV_USD_C,WALK_FLAG_C,NO_SHOW_FLAG_C,DUP_INDEX_C,CONFIRMATION_NUM_R,CONFIRMATION_PAGE_R,RESERVATION_DATE_R,ARRIVAL_FLG_R,ARRIVAL_DATE_R,DEPARTURE_DATE_R,DIRECT_NIGHTS_NUM_R,RESERVATION_STATUS_R,ENTRY_TIME_R,ENTRY_HOTEL_CODE_R,LAST_CHANGE_DATE_R,ROOM_TYPE_CODE_R,ROOM_TYPE_DESCRIPTION_R,MAJOR_MARKET_CODE_R,PMS_RATE_CATEGORY_CODE_R,NUM_ROOMS_R,ADULT_NUM_R,CHILDREN_NUM_R,NT_RATE_R,STATE_R,COUNTRY_CODE_R,ETA_R,FLIGHTT_INFO_R,LENGTH_OF_STAY_R,LENGTH_OF_STAY_CATEGORY_R,CALCULATED_NIGHTS_NUM_R,ROOM_NIGHTS_R,STATUS_CALCULATION_R,GROUPS_VS_FIT_R,GUEST_COUNTRY_R,GOLDPASSPORT_FLG_R,MEMBER_STATUS_R,PACE_CATEGORY_R,PACE_R,OFFER_FLG_R,PAST_VS_FUTURE_R,REVENUE_R,REVENUE_USD_R,CHANNEL_CODE_R,e_delivereddate_I,e_delivereddate_adj_I,e_status_I,e_hy_gss_tier_I,e_hy_gss_conf_num_I,e_hy_gss_conf_page_I,e_hy_feedback_type_I,e_hy_gss_rate_plan_code_I,e_country_I,e_hy_gss_check_in_by_I,e_hy_gss_check_out_by_I,e_hy_gss_marketing_emails_ok_yn_I,e_hy_gss_promo_emails_ok_yn_I,e_hy_gss_club_access_yn_I,e_hy_gss_gender_I,e_hy_gss_title_text_I,e_hy_gss_language_I,e_hy_gss_room_floor_I,e_hy_gss_check_in_time_text_I,e_hy_gss_check_out_time_text_I,e_hy_gss_check_in_time_I,e_hy_gss_check_out_time_I,e_checkin_I,e_checkout_I,a_last_seen_page_name_I,a_last_submitted_page_name_I,Survey_ID_H,Response_Date_H,Guest_Checkin_Date_H,Guest_Checkout_Date_H,Length_Stay_H,Guest_State_H,Guest_Country_H,Gender_H,Age_Range_H,POV_H,Language_H,DOE_H,Booking_Location_H,Num_Adults_H,Num_Kids_H,Num_Rooms_H,Conf_Num_Orig_H,Conf_Num_H,Conf_Page_H,Rate_Plan_H,Gross_Rev_H,Net_Rev_H,Room_Rev_H,CC_Type_H,Currency_H,Clublounge_Used_H,Spa_Used_H,GDS_Source_H,Checkin_Length_H,Room_Num_H,Room_Type_H,GP_Tier_H,Status_H,Feedback_Type_H,Mobile_First_H,Mobile_H,Overall_Sat_H,Guest_Room_H,Tranquility_H,Condition_Hotel_H,Customer_SVC_H,Staff_Cared_H,Internet_Sat_H,Check_In_H,FB_FREQ_H,FB_Overall_Experience_H,Internet_Dissat_Lobby_H,Internet_Dissat_Slow_H,Internet_Dissat_Expensive_H,Internet_Dissat_Connectivity_H,Internet_Dissat_Billing_H,Internet_Dissat_Wired_H,Internet_Dissat_Other_H,TV_Internet_General_H,Room_Dissat_Internet_H,eff_date_CC,avail_room_cnt_CC,occ_room_cnt_CC,average_daily_rate_CC,Spirit_PL,Property_ID_PL,Hotel_Name_Long_PL,Hotel_Name_Short_PL,Award_Category_PL,Status_PL,City_PL,State_PL,US_Region_PL,Postal_Code_PL,Country_PL,Ops_Region_PL,Sub_Continent_PL,Property_DMA_PL,Property_Latitude_PL,Property_Longitude_PL,Currency_PL,Dom_Intl_PL,Guest_NPS_Goal_PL,STR_Number_PL,STR_Market_PL,Brand_PL,Brand_Initial_PL,Club_Type_PL,Hotel_Inventory_PL,Floors_PL,Total_Meeting_Space_PL,Union_PL,GRegion_PL,DRegion_PL,Region_PL,Category_PL,Scope_of_Service_PL,Type_PL,Class_PL,Location_PL,Bucket_PL,Relationship_PL,All_Suites_PL,Bell_Staff_PL,Boutique_PL,Business_Center_PL,Casino_PL,Conference_PL,Convention_PL,Dry_Cleaning_PL,Elevators_PL,Fitness_Center_PL,Fitness_Trainer_PL,Golf_PL,Indoor_Corridors_PL,Laundry_PL,Limo_Service_PL,Mini_Bar_PL,Pool_Indoor_PL,Pool_Outdoor_PL,Regency_Grand_Club_PL,Resort_PL,Restaurant_PL,Self_Parking_PL,Shuttle_Service_PL,Ski_PL,Spa_PL,Spa_fitness_center_PL,Spa_online_booking_PL,Spa_offering_PL,Valet_Parking_PL,GP_Tier,Sub_Channel_Category,Channel_Category,Booking_Channel,NPS_Type,Stay_Sequence,Stay_Sequence_Brand,Completed_Survey_Sequence,Days_Since_Last_Stay,Days_Until_Next_Stay'''
print('Total Number of columns: %d' %(len(columns_all.split(','))))

columns_pred = '''Likelihood_Recommend_H,ROOM_NUM_C,LENGTH_OF_STAY_C,NUMBER_OF_ROOMS_C,ADULT_NUM_C,CHILDREN_NUM_C,QUOTED_RATE_C,PMS_ROOM_REV_C,PMS_ROOM_REV_USD_C,PMS_TOTAL_REV_C,PMS_TOTAL_REV_USD_C,PMS_FOOD_BEVERAGE_REV_C,PMS_FOOD_BEVERAGE_REV_USD_C,PMS_OTHER_REV_C,PMS_OTHER_REV_USD_C,DUP_INDEX_C,CONFIRMATION_NUM_R,CONFIRMATION_PAGE_R,ARRIVAL_FLG_R,DIRECT_NIGHTS_NUM_R,NUM_ROOMS_R,ADULT_NUM_R,CHILDREN_NUM_R,NT_RATE_R,LENGTH_OF_STAY_R,CALCULATED_NIGHTS_NUM_R,ROOM_NIGHTS_R,PACE_R,REVENUE_R,REVENUE_USD_R,e_hy_gss_conf_num_I,e_hy_gss_conf_page_I,Length_Stay_H,Num_Adults_H,Num_Kids_H,Num_Rooms_H,Conf_Num_Orig_H,Conf_Num_H,Conf_Page_H,Gross_Rev_H,Net_Rev_H,Room_Rev_H,Checkin_Length_H,Room_Num_H,Overall_Sat_H,Guest_Room_H,Tranquility_H,Condition_Hotel_H,Customer_SVC_H,Staff_Cared_H,Internet_Sat_H,Check_In_H,FB_FREQ_H,FB_Overall_Experience_H,Internet_Dissat_Lobby_H,Internet_Dissat_Slow_H,Internet_Dissat_Expensive_H,Internet_Dissat_Connectivity_H,Internet_Dissat_Billing_H,Internet_Dissat_Wired_H,Internet_Dissat_Other_H,TV_Internet_General_H,Room_Dissat_Internet_H,avail_room_cnt_CC,occ_room_cnt_CC,average_daily_rate_CC,Guest_NPS_Goal_PL,STR_Number_PL,Hotel_Inventory_PL,Floors_PL,Total_Meeting_Space_PL,Stay_Sequence,Stay_Sequence_Brand,Completed_Survey_Sequence,Days_Since_Last_Stay,Days_Until_Next_Stay'''
print('Columns used for predict: %d' %(len(columns_pred.split(','))))

Total Number of columns: 237
Columns used for predict: 76


In [11]:
query_all = '''SELECT {} FROM hyatt_all where nps_type IN ('Promoter','Detractor','Passive') AND us_region_pl IN ('West') order by checkout_header_id_c'''.format(columns_all)

query_pred = '''SELECT {} FROM hyatt_all where nps_type IN ('Promoter','Detractor','Passive') AND us_region_pl IN ('West') order by checkout_header_id_c'''.format(columns_pred)


Extract data on GPU

In [12]:
print('Query_pred is: ',query_pred)

results_pred = client.sql_execute_gpudf(session,query_pred,device_id=0,first_n=-1)

Query_pred is:  SELECT Likelihood_Recommend_H,ROOM_NUM_C,LENGTH_OF_STAY_C,NUMBER_OF_ROOMS_C,ADULT_NUM_C,CHILDREN_NUM_C,QUOTED_RATE_C,PMS_ROOM_REV_C,PMS_ROOM_REV_USD_C,PMS_TOTAL_REV_C,PMS_TOTAL_REV_USD_C,PMS_FOOD_BEVERAGE_REV_C,PMS_FOOD_BEVERAGE_REV_USD_C,PMS_OTHER_REV_C,PMS_OTHER_REV_USD_C,DUP_INDEX_C,CONFIRMATION_NUM_R,CONFIRMATION_PAGE_R,ARRIVAL_FLG_R,DIRECT_NIGHTS_NUM_R,NUM_ROOMS_R,ADULT_NUM_R,CHILDREN_NUM_R,NT_RATE_R,LENGTH_OF_STAY_R,CALCULATED_NIGHTS_NUM_R,ROOM_NIGHTS_R,PACE_R,REVENUE_R,REVENUE_USD_R,e_hy_gss_conf_num_I,e_hy_gss_conf_page_I,Length_Stay_H,Num_Adults_H,Num_Kids_H,Num_Rooms_H,Conf_Num_Orig_H,Conf_Num_H,Conf_Page_H,Gross_Rev_H,Net_Rev_H,Room_Rev_H,Checkin_Length_H,Room_Num_H,Overall_Sat_H,Guest_Room_H,Tranquility_H,Condition_Hotel_H,Customer_SVC_H,Staff_Cared_H,Internet_Sat_H,Check_In_H,FB_FREQ_H,FB_Overall_Experience_H,Internet_Dissat_Lobby_H,Internet_Dissat_Slow_H,Internet_Dissat_Expensive_H,Internet_Dissat_Connectivity_H,Internet_Dissat_Billing_H,Internet_Dissat_Wi

Extract data on CPU

In [13]:
def getDataAll(query):
    print('Query_all is: ',query_all)
    
    print('after no')
    results_all = client.sql_execute(session,query,True,None,-1)
    #results.row_set.columns[0].nulls
    #results.row_set.row_desc
    
    return results_all

In [14]:
from multiprocessing.pool import ThreadPool

pool = ThreadPool(processes=4)
data_thread = pool.apply_async(getDataAll, args = (query_all,))
pool.close()

Query_all is:  SELECT Likelihood_Recommend_H,CHECKOUT_HEADER_ID_C,MARKET_CODE_C,MARKET_DESC_C,MARKET_GROUP_C,MAJOR_MARKET_CODE_C,CHANNEL_CODE_C,RATE_PLAN_C,SPLIT_RATE_PLAN_C,CONS_GUEST_ID_C,ROOM_NUM_C,ROOM_TYPE_CODE_C,ROOM_TYPE_DESCRIPTION_C,WALK_IN_FLG_C,RESERVATION_CONFIRMATION_NUM_C,RESERVATION_CONFIRMATION_PAGE_C,RESERVATION_STATUS_C,CHECK_IN_DATE_C,CHECK_OUT_DATE_C,LENGTH_OF_STAY_C,NUMBER_OF_ROOMS_C,ADULT_NUM_C,CHILDREN_NUM_C,POV_CODE_C,QUOTED_RATE_C,PMS_ROOM_REV_C,PMS_ROOM_REV_USD_C,PMS_TOTAL_REV_C,PMS_TOTAL_REV_USD_C,PMS_FOOD_BEVERAGE_REV_C,PMS_FOOD_BEVERAGE_REV_USD_C,PMS_OTHER_REV_C,PMS_OTHER_REV_USD_C,WALK_FLAG_C,NO_SHOW_FLAG_C,DUP_INDEX_C,CONFIRMATION_NUM_R,CONFIRMATION_PAGE_R,RESERVATION_DATE_R,ARRIVAL_FLG_R,ARRIVAL_DATE_R,DEPARTURE_DATE_R,DIRECT_NIGHTS_NUM_R,RESERVATION_STATUS_R,ENTRY_TIME_R,ENTRY_HOTEL_CODE_R,LAST_CHANGE_DATE_R,ROOM_TYPE_CODE_R,ROOM_TYPE_DESCRIPTION_R,MAJOR_MARKET_CODE_R,PMS_RATE_CATEGORY_CODE_R,NUM_ROOMS_R,ADULT_NUM_R,CHILDREN_NUM_R,NT_RATE_R,STATE_R,COUN

Create handle for GPU pointer

In [15]:
from numba import cuda
from numba.cuda.cudadrv import drvapi


In [16]:
gpu_handle = drvapi.cu_ipc_mem_handle(*results_pred.df_handle)
ipc_handle = cuda.driver.IpcHandle(None,gpu_handle,results_pred.df_size)
contxt = cuda.current_context()

ipc = ipc_handle.open(contxt)
pprint(ipc)
pprint(cuda.current_context().get_memory_info())

_MemoryInfo(free=22380150784, total=25630801920)


ipc is space in the GPU memory containing data

Converting ipc into NDArray

In [17]:
import numpy as np

In [18]:
type = np.dtype(np.byte)
data_arr = cuda.devicearray.DeviceNDArray(shape = ipc.size, strides = type.itemsize,\
                                         dtype = type, gpu_data = ipc) 

Reading data using GPU Arrow

In [19]:
from pygdf.gpuarrow import GpuArrowReader

In [20]:
reader = GpuArrowReader(data_arr)

Wrapping result in PYGDF

In [21]:
from pygdf.dataframe import DataFrame

In [22]:
df = DataFrame(reader.to_dict().items())

Overview of dataset

In [23]:
df

  Likelihood_Recommend_H ROOM_NUM_C LENGTH_OF_STAY_C NUMBER_OF_ROOMS_C ADULT_NUM_C CHILDREN_NUM_C QUOTED_RATE_C ... PMS_ROOM_REV_C
0                      6      339.0              3.0               1.0         2.0            2.0         134.0 ...          402.0
1                     10     1432.0              2.0               1.0         1.0                        209.0 ...          418.0
2                      9     3360.0              1.0               1.0         2.0                        306.0 ...          306.0
3                      7      233.0              4.0               1.0         2.0            2.0         42.74 ...         170.96
4                      9     2254.0              7.0               1.0         2.0            2.0         310.0 ...         2170.0
[186998 more rows]
[68 more columns]

### Data Preprocessing

In [24]:
predict_column = set(['Likelihood_Recommend_H'])
feature_columns = set(df.columns) - predict_column
var_num = set()
var_cat = set()

separate cat and num columns

In [25]:
unique_val = {}
for col in feature_columns:
    try:
        val = df[col].unique_k(999)
        unique_val[col] = val
    except ValueError:
        #more than 999 unique values
        var_num.add(col)
    else:
        #value less than 999
        value = len(val)
        if value <= 1:
            del df[col]
        elif 1<value<999:
            var_cat.add(col)
        else:
            var_num.add(col)
            
print('Number of numerical columns: {}'.format(len(var_num)))
print('Number of categorical columns: {}'.format(len(var_cat)))

Number of numerical columns: 24
Number of categorical columns: 39


Replace Null values with mean and scale for num columns

In [26]:
#change dtype to float 64
for col in df.columns:
    df[col] = df[col].astype(np.float64)

In [27]:
for col in var_num:
    df[col] = df[col].fillna(df[col].mean())
    assert df[col].null_count == 0
    std = df[col].std()
    # drop near constant columns
    if not np.isfinite(std) or std < 1e-4:
        del df[col]
        print('drop near constant', col)
    else:
        df[col] = df[col].scale()

One-hot encode cat columns

In [28]:
for col in var_cat:
    cats = unique_val[col][1:]  # drop first
    df = df.one_hot_encoding(col, prefix=col, cats=cats)
    del df[col]

In [29]:
nrows = len(df) #total no of rows

In [30]:
#Ensure dtypes are in float 64
{df[k].dtype for k in df.columns}
df['intercept'] = np.ones(nrows, dtype=np.float64)

Divide training and testing datasets into 70:30

In [31]:
cp = int(.70 * nrows)

df_train, df_test = df.loc[:cp-1], df.loc[cp:]

print('Total number of rows: %d' %(nrows))
print('Training datsets has: %d' %(len(df_train)))
print('Test datsets has: %d' %(len(df_test)))

Total number of rows: 187003
Training datsets has: 130902
Test datsets has: 56101


Convert data frames into matrices

In [32]:
train_data_mat = df_train.as_gpu_matrix(columns=df.columns[1:])
train_result_mat = df_train.as_gpu_matrix(columns=[df.columns[0]])
test_data_mat = df_test.as_gpu_matrix(columns=df.columns[1:])
test_result_mat = df_test.as_gpu_matrix(columns=[df.columns[0]])

Close IPC handle as matrix copies are created

In [33]:
ipc_handle.close()

ctype pointers to GPU matrices

In [34]:
train_data_mat_ptr = train_data_mat.device_ctypes_pointer
train_result_mat_ptr = train_result_mat.device_ctypes_pointer
print('train_data_mat_ptr address', hex(train_data_mat_ptr.value))
print('train_result_mat_ptr address', hex(train_result_mat_ptr.value))

test_data_mat_ptr = test_data_mat.device_ctypes_pointer
test_result_mat_ptr = test_result_mat.device_ctypes_pointer
print('test_data_mat_ptr address', hex(test_data_mat_ptr.value))
print('test_result_mat_ptr address', hex(test_result_mat_ptr.value))

train_data_mat_ptr address 0x10723200000
train_result_mat_ptr address 0x107d1a5fc00
test_data_mat_ptr address 0x107d1c00000
test_result_mat_ptr address 0x1081c8bc200


In [35]:
print(train_data_mat.shape)
print(train_result_mat.shape)
print(test_data_mat.shape)
print(test_result_mat.shape)

(130902, 2796)
(130902, 1)
(56101, 2796)
(56101, 1)


Convert data frames into matrices

In [36]:
mtime = time()
#train_data_mat = df_train.as_matrix(columns=df.columns[1:])
#train_result_mat = df_train.as_matrix(columns=[df.columns[0]])
#test_data_mat = df_test.as_matrix(columns=df.columns[1:])
#test_result_mat = df_test.as_matrix(columns=[df.columns[0]])
metime = time()

matrices to numpy arrays

In [37]:
#train_data_np = np.array(train_data_mat,dtype = np.float64)
#train_result_np = np.array(train_result_mat,dtype = np.float64)
#test_data_np = np.array(test_data_mat,dtype = np.float64)
#test_result_np = np.array(test_result_mat,dtype = np.float64)

In [38]:
#print(train_data_np.shape)
#rint(train_result_np.shape)
#print(test_data_np.shape)
#print(test_result_np.shape)

### Linear regression using H2oaiglm

In [42]:
#Load h2oaiglm
import h2oaiglm
#from h2oaiglm.solvers.kmeans_gpu import KMeansGPU2
from ctypes import *
import pandas as pd

In [40]:
xtrain=c_void_p(train_data_mat_ptr.value)
ytrain=c_void_p(train_result_mat_ptr.value)
xtest=c_void_p(test_data_mat_ptr.value)
ytest=c_void_p(test_result_mat_ptr.value)

Select Data size and algorithm to process

In [43]:
n=train_data_mat.shape[1]
mTrain=train_data_mat.shape[0]
mValid=test_data_mat.shape[0]

print("No. of Features=%d mTrain=%d mValid=%d" % (n,mTrain,mValid))
# Order of data
fortran = 1
print("fortran=%d" % (fortran))

No. of Features=2796 mTrain=130902 mValid=56101
fortran=1


Define function to use PYGDF data pointers in GPU

In [47]:
def RunH2Oaiglm(arg):
    intercept,standardize, lambda_min_ratio, nFolds, nAlphas, nLambdas, nGPUs = arg
    
        # set solver cpu/gpu according to input args
    if((nGPUs>0) and (h2oaiglm.ElasticNetSolverGPU is None)):
        print("\nGPU solver unavailable, using CPU solver\n")
        nGPUs=0

    sharedA = 0
    sourceme = 0
    sourceDev = 0
    nThreads = 1 if(nGPUs==0) else nGPUs # not required number of threads, but normal.

    print("Setting up Solver")
    Solver = h2oaiglm.ElasticNetSolverGPU if(nGPUs>0) else h2oaiglm.ElasticNetSolverCPU
    print(Solver)


    #  Solver = h2oaiglm.ElasticNetSolverCPU
    assert Solver != None, "Couldn't instantiate ElasticNetSolver"
    enet = Solver(sharedA, nThreads, nGPUs, 'c' if fortran else 'r', intercept, standardize, \
                  lambda_min_ratio, nLambdas, nFolds, nAlphas)

    # Not using weights
    #e = c_vod_p(0)

    print("Solving")
    ## Solve
    #t0 = time.time()
    #print("vars: %d %d %d %d %d %d %d" % (sourceDev, mTrain, n, mValid, intercept, standardize, precision))
    #enet.fit(sourceDev, mTrain, n, mValid, intercept, standardize, precision, a, b, c, d, e)
    mod = enet.fit(xtrain, ytrain)
    #t1 = time.time()
    print("Done Solving")
    
    #Display most important metrics
    print('Alphas: ',len(enet.getalphas()))
    al = enet.getalphas()
    print('Lambdas: ',(enet.getlambdas()))
    print('Tols: ',(enet.gettols()))
    print('RMSE: ',(enet.getrmse()))
    
    print('Predicting')
    pred = enet.predict(xtest)
    print('Prediction complete')
    
    return al,mod,pred

Setting up parameters

In [48]:
intercept = 1 #
standardize = 0
lambda_min_ratio=1E-9
nFolds=5
nAlphas=4
nLambdas=100

import subprocess
maxNGPUS = int(subprocess.check_output("nvidia-smi -L | wc -l", shell=True))
print("Maximum Number of GPUS:", maxNGPUS)
nGPUs=maxNGPUS # choose all GPUs

Maximum Number of GPUS: 4


Run the model to predict

In [49]:
arg = intercept,standardize, lambda_min_ratio, nFolds, nAlphas, nLambdas, nGPUs 
ptime = time()
al,mod,pred_val = RunH2Oaiglm(arg)
petime = time() 

Setting up Solver
<class 'h2oaiglm.solvers.elastic_net_gpu.ElasticNetSolverGPU'>
Solving


IndexError: tuple index out of range

In [ ]:
print(mod.shape)
print(pred_val.shape)
#np.ones((pred_val.shape[0],1),dtype=pred_val.dtype)

In [ ]:
al[0][2]

Join predection to Original Data Frame

In [ ]:
df_pred = pd.DataFrame(pred_val[np.newaxis][0].T)
#df_pred.columns = ['pred_recommend_rating']
#df_pred

In [ ]:
print('Waiting for CPU result')
pool.join()
print('Done waiting')
df_org = pd.DataFrame(getDict(data_thread.get()))
df_org_test = df_org[cp:]
df_org_test = df_org_test.reset_index(drop = True)
df_org_test['eff_date_CC'].replace(-9223372036854775808,1,inplace=True)

df_org_test = df_org_test[columns_all.split(',')]  #arrange dataframe according to mapd table layout
#len(df_org_test)

df_org_test['pred_recommend_rating'] = df_pred    #join pred result
#df_org_test

Exporting the dataframe to csv file

In [ ]:
import csv

In [ ]:
csv_time = time()

os.system("rm -f hyatt_predict1.csv;")
export_path = os.path.join(PWD[0],'hyatt_predict1.csv')
export_path

df_org_test.to_csv(export_path,sep = ',',index = False,quoting=csv.QUOTE_NONNUMERIC)
#h2o.export_file(hf_org_test, path = export_path)

csve_time = time()

Create a temp table in MapD and load prediction data

In [ ]:
drop_query = '''DROP TABLE IF EXISTS hyatt_all_pred_3;'''

temp_query = '''CREATE TABLE hyatt_all_pred_3 (Likelihood_Recommend_H integer,
  CHECKOUT_HEADER_ID_C float,
  MARKET_CODE_C text encoding dict,
  MARKET_DESC_C text encoding dict,
  MARKET_GROUP_C text encoding dict,
  MAJOR_MARKET_CODE_C text encoding dict,
  CHANNEL_CODE_C text encoding dict,
  RATE_PLAN_C text encoding dict,
  SPLIT_RATE_PLAN_C text encoding dict,
  CONS_GUEST_ID_C float,
  ROOM_NUM_C float,
  ROOM_TYPE_CODE_C text encoding dict,
  ROOM_TYPE_DESCRIPTION_C text encoding dict,
  WALK_IN_FLG_C text encoding dict,
  RESERVATION_CONFIRMATION_NUM_C float,
  RESERVATION_CONFIRMATION_PAGE_C float,
  RESERVATION_STATUS_C text encoding dict,
  CHECK_IN_DATE_C date,
  CHECK_OUT_DATE_C date,
  LENGTH_OF_STAY_C float,
  NUMBER_OF_ROOMS_C float,
  ADULT_NUM_C float,
  CHILDREN_NUM_C float,
  POV_CODE_C text encoding dict,
  QUOTED_RATE_C float,
  PMS_ROOM_REV_C float,
  PMS_ROOM_REV_USD_C float,
  PMS_TOTAL_REV_C float,
  PMS_TOTAL_REV_USD_C float,
  PMS_FOOD_BEVERAGE_REV_C float,
  PMS_FOOD_BEVERAGE_REV_USD_C float,
  PMS_OTHER_REV_C float,
  PMS_OTHER_REV_USD_C float,
  WALK_FLAG_C text encoding dict,
  NO_SHOW_FLAG_C text encoding dict,
  DUP_INDEX_C float,
  CONFIRMATION_NUM_R float,
  CONFIRMATION_PAGE_R float,
  RESERVATION_DATE_R date,
  ARRIVAL_FLG_R float,
  ARRIVAL_DATE_R date,
  DEPARTURE_DATE_R date,
  DIRECT_NIGHTS_NUM_R float,
  RESERVATION_STATUS_R text encoding dict,
  ENTRY_TIME_R text encoding dict,
  ENTRY_HOTEL_CODE_R text encoding dict,
  LAST_CHANGE_DATE_R timestamp,
  ROOM_TYPE_CODE_R text encoding dict,
  ROOM_TYPE_DESCRIPTION_R text encoding dict,
  MAJOR_MARKET_CODE_R text encoding dict,
  PMS_RATE_CATEGORY_CODE_R text encoding dict,
  NUM_ROOMS_R float,
  ADULT_NUM_R float,
  CHILDREN_NUM_R float,
  NT_RATE_R float,
  STATE_R text encoding dict,
  COUNTRY_CODE_R text encoding dict,
  ETA_R text encoding dict,
  FLIGHTT_INFO_R text encoding dict,
  LENGTH_OF_STAY_R float,
  LENGTH_OF_STAY_CATEGORY_R text encoding dict,
  CALCULATED_NIGHTS_NUM_R float,
  ROOM_NIGHTS_R float,
  STATUS_CALCULATION_R text encoding dict,
  GROUPS_VS_FIT_R text encoding dict,
  GUEST_COUNTRY_R text encoding dict,
  GOLDPASSPORT_FLG_R text encoding dict,
  MEMBER_STATUS_R text encoding dict,
  PACE_CATEGORY_R text encoding dict,
  PACE_R float,
  OFFER_FLG_R text encoding dict,
  PAST_VS_FUTURE_R text encoding dict,
  REVENUE_R float,
  REVENUE_USD_R float,
  CHANNEL_CODE_R text encoding dict,
  e_delivereddate_I timestamp,
  e_delivereddate_adj_I timestamp,
  e_status_I text encoding dict,
  e_hy_gss_tier_I text encoding dict,
  e_hy_gss_conf_num_I float,
  e_hy_gss_conf_page_I float,
  e_hy_feedback_type_I text encoding dict,
  e_hy_gss_rate_plan_code_I text encoding dict,
  e_country_I text encoding dict,
  e_hy_gss_check_in_by_I text encoding dict,
  e_hy_gss_check_out_by_I text encoding dict,
  e_hy_gss_marketing_emails_ok_yn_I text encoding dict,
  e_hy_gss_promo_emails_ok_yn_I text encoding dict,
  e_hy_gss_club_access_yn_I text encoding dict,
  e_hy_gss_gender_I text encoding dict,
  e_hy_gss_title_text_I text encoding dict,
  e_hy_gss_language_I text encoding dict,
  e_hy_gss_room_floor_I text encoding dict,
  e_hy_gss_check_in_time_text_I text encoding dict,
  e_hy_gss_check_out_time_text_I text encoding dict,
  e_hy_gss_check_in_time_I text encoding dict,
  e_hy_gss_check_out_time_I text encoding dict,
  e_checkin_I date,
  e_checkout_I date,
  a_last_seen_page_name_I text encoding dict,
  a_last_submitted_page_name_I text encoding dict,
  Survey_ID_H float,
  Response_Date_H timestamp,
  Guest_Checkin_Date_H date,
  Guest_Checkout_Date_H date,
  Length_Stay_H float,
  Guest_State_H text encoding dict,
  Guest_Country_H text encoding dict,
  Gender_H text encoding dict,
  Age_Range_H text encoding dict,
  POV_H text encoding dict,
  Language_H text encoding dict,
  DOE_H date,
  Booking_Location_H text encoding dict,
  Num_Adults_H float,
  Num_Kids_H float,
  Num_Rooms_H float,
  Conf_Num_Orig_H float,
  Conf_Num_H float,
  Conf_Page_H float,
  Rate_Plan_H text encoding dict,
  Gross_Rev_H float,
  Net_Rev_H float,
  Room_Rev_H float,
  CC_Type_H text encoding dict,
  Currency_H text encoding dict,
  Clublounge_Used_H text encoding dict,
  Spa_Used_H text encoding dict,
  GDS_Source_H text encoding dict,
  Checkin_Length_H float,
  Room_Num_H float,
  Room_Type_H text encoding dict,
  GP_Tier_H text encoding dict,
  Status_H text encoding dict,
  Feedback_Type_H text encoding dict,
  Mobile_First_H text encoding dict,
  Mobile_H text encoding dict,
  Overall_Sat_H float,
  Guest_Room_H float,
  Tranquility_H float,
  Condition_Hotel_H float,
  Customer_SVC_H float,
  Staff_Cared_H float,
  Internet_Sat_H float,
  Check_In_H float,
  FB_FREQ_H float,
  FB_Overall_Experience_H float,
  Internet_Dissat_Lobby_H float,
  Internet_Dissat_Slow_H float,
  Internet_Dissat_Expensive_H float,
  Internet_Dissat_Connectivity_H float,
  Internet_Dissat_Billing_H float,
  Internet_Dissat_Wired_H float,
  Internet_Dissat_Other_H float,
  TV_Internet_General_H float,
  Room_Dissat_Internet_H float,
  eff_date_CC date,
  avail_room_cnt_CC float,
  occ_room_cnt_CC float,
  average_daily_rate_CC float,
  Spirit_PL text encoding dict,
  Property_ID_PL text encoding dict,
  Hotel_Name_Long_PL text encoding dict,
  Hotel_Name_Short_PL text encoding dict,
  Award_Category_PL text encoding dict,
  Status_PL text encoding dict,
  City_PL text encoding dict,
  State_PL text encoding dict,
  US_Region_PL text encoding dict,
  Postal_Code_PL text encoding dict,
  Country_PL text encoding dict,
  Ops_Region_PL text encoding dict,
  Sub_Continent_PL text encoding dict,
  Property_DMA_PL text encoding dict,
  Property_Latitude_PL float,
  Property_Longitude_PL float,
  Currency_PL text encoding dict,
  Dom_Intl_PL text encoding dict,
  Guest_NPS_Goal_PL float,
  STR_Number_PL float,
  STR_Market_PL text encoding dict,
  Brand_PL text encoding dict,
  Brand_Initial_PL text encoding dict,
  Club_Type_PL text encoding dict,
  Hotel_Inventory_PL float,
  Floors_PL float,
  Total_Meeting_Space_PL float,
  Union_PL text encoding dict,
  GRegion_PL text encoding dict,
  DRegion_PL text encoding dict,
  Region_PL text encoding dict,
  Category_PL text encoding dict,
  Scope_of_Service_PL text encoding dict,
  Type_PL text encoding dict,
  Class_PL text encoding dict,
  Location_PL text encoding dict,
  Bucket_PL text encoding dict,
  Relationship_PL text encoding dict,
  All_Suites_PL text encoding dict,
  Bell_Staff_PL text encoding dict,
  Boutique_PL text encoding dict,
  Business_Center_PL text encoding dict,
  Casino_PL text encoding dict,
  Conference_PL text encoding dict,
  Convention_PL text encoding dict,
  Dry_Cleaning_PL text encoding dict,
  Elevators_PL text encoding dict,
  Fitness_Center_PL text encoding dict,
  Fitness_Trainer_PL text encoding dict,
  Golf_PL text encoding dict,
  Indoor_Corridors_PL text encoding dict,
  Laundry_PL text encoding dict,
  Limo_Service_PL text encoding dict,
  Mini_Bar_PL text encoding dict,
  Pool_Indoor_PL text encoding dict,
  Pool_Outdoor_PL text encoding dict,
  Regency_Grand_Club_PL text encoding dict,
  Resort_PL text encoding dict,
  Restaurant_PL text encoding dict,
  Self_Parking_PL text encoding dict,
  Shuttle_Service_PL text encoding dict,
  Ski_PL text encoding dict,
  Spa_PL text encoding dict,
  Spa_fitness_center_PL text encoding dict,
  Spa_online_booking_PL text encoding dict,
  Spa_offering_PL text encoding dict,
  Valet_Parking_PL text encoding dict,
  GP_Tier text encoding dict,
  Sub_Channel_Category text encoding dict,
  Channel_Category text encoding dict,
  Booking_Channel text encoding dict,
  NPS_Type text encoding dict,
  Stay_Sequence float,
  Stay_Sequence_Brand float,
  Completed_Survey_Sequence float,
  Days_Since_Last_Stay float,
  Days_Until_Next_Stay float,
  pred_recommend_rating Integer);'''

load_query = "COPY hyatt_all_pred_3 FROM '/raidStorage/wamsi/mapd-ml/notebooks/hyatt_predict1.csv';"

In [ ]:
#create table

try:                                    #if the table does not exist
    client.sql_execute(session,temp_query,True,None,first_n = -1)

except:                                 #if table exists
    client.sql_execute(session,drop_query,True,None,first_n = -1)
    client.sql_execute(session,temp_query,True,None,first_n = -1)

In [ ]:
#load data
load = client.sql_execute(session,load_query,True,None,first_n = -1)
print('Records {}'.format(load.row_set.columns[0].data.str_col[0]))

In [ ]:
print('Total time for execution: %d' %((time()-start_time)))
print('Time for making matrices: %d' %(metime - mtime))
print('Time for predicting: %d' %(petime - ptime))

#print('Time taken for extracting data: %d'%((qse_time-qs_time)))
#print('Time for creating dataframe %d'%(dfe_time-df_time))
#print('Preprocessing time: %d'%((dpe_time-dp_time)))
#print('Time to predict: %d'%((he_time-h_time)))
#print('Time to write to csv: %d'%(csve_time-csv_time))

Start MapD immerse

In [ ]:
%%HTML
<a href="http://localhost:9999/" target="_blank">MapD Immerse</a>